In [221]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sklearn
%matplotlib inline
print('Done!')

Done!


In [222]:
import matplotlib.pyplot as plt
import seaborn as sns
sns.set_style('darkgrid')

from sklearn.neighbors import KNeighborsRegressor
import scipy.stats
from sklearn.preprocessing import StandardScaler
from pycaret.regression import setup, compare_models
from sklearn.model_selection import KFold, cross_val_score

from catboost import CatBoostRegressor
from sklearn.linear_model import BayesianRidge, HuberRegressor, Ridge, OrthogonalMatchingPursuit
from lightgbm import LGBMRegressor
from sklearn.ensemble import GradientBoostingRegressor
from xgboost import XGBRegressor

import optuna

In [223]:
spaceship_train_data = pd.read_csv('C:\\Users\\Auditor\\Desktop\\Victor\\New folder\\Data Science\\Projects\\Spaceship Titanic\\train.csv')

In [224]:
spaceship_test_data = pd.read_csv('C:\\Users\\Auditor\\Desktop\\Victor\\New folder\\Data Science\\Projects\\Spaceship Titanic\\test.csv')

In [225]:
spaceship_train_data

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [226]:
spaceship_test_data

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,G/3/S,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,F/4/S,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,C/0/S,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,C/1/S,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,F/5/S,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,Earth,True,G/1496/S,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,0.0,0.0,Jeron Peter
4273,9269_01,Earth,False,NaN,TRAPPIST-1e,42.0,False,0.0,847.0,17.0,10.0,144.0,Matty Scheron
4274,9271_01,Mars,True,D/296/P,55 Cancri e,NaN,False,0.0,0.0,0.0,0.0,0.0,Jayrin Pore
4275,9273_01,Europa,False,D/297/P,NaN,NaN,False,0.0,2680.0,0.0,0.0,523.0,Kitakan Conale


In [227]:
spaceship_train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8693 entries, 0 to 8692
Data columns (total 14 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   8693 non-null   object 
 1   HomePlanet    8492 non-null   object 
 2   CryoSleep     8476 non-null   object 
 3   Cabin         8494 non-null   object 
 4   Destination   8511 non-null   object 
 5   Age           8514 non-null   float64
 6   VIP           8490 non-null   object 
 7   RoomService   8512 non-null   float64
 8   FoodCourt     8510 non-null   float64
 9   ShoppingMall  8485 non-null   float64
 10  Spa           8510 non-null   float64
 11  VRDeck        8505 non-null   float64
 12  Name          8493 non-null   object 
 13  Transported   8693 non-null   bool   
dtypes: bool(1), float64(6), object(7)
memory usage: 891.5+ KB


In [228]:
spaceship_test_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4277 entries, 0 to 4276
Data columns (total 13 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   PassengerId   4277 non-null   object 
 1   HomePlanet    4190 non-null   object 
 2   CryoSleep     4184 non-null   object 
 3   Cabin         4177 non-null   object 
 4   Destination   4185 non-null   object 
 5   Age           4186 non-null   float64
 6   VIP           4184 non-null   object 
 7   RoomService   4195 non-null   float64
 8   FoodCourt     4171 non-null   float64
 9   ShoppingMall  4179 non-null   float64
 10  Spa           4176 non-null   float64
 11  VRDeck        4197 non-null   float64
 12  Name          4183 non-null   object 
dtypes: float64(6), object(7)
memory usage: 434.5+ KB


In [229]:
#target 
last_column = spaceship_train_data['Transported']
test_ids = spaceship_test_data['PassengerId']

In [230]:
spaceship_train_data1 = spaceship_train_data.drop(['PassengerId','Cabin','Name','Transported'], axis=1)
spaceship_test_data1 = spaceship_test_data.drop(['PassengerId','Cabin','Name'], axis=1)

In [231]:
spaceship_train_data1

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0
1,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0
2,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0
3,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0
4,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0
...,...,...,...,...,...,...,...,...,...,...
8688,Europa,False,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0
8689,Earth,True,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0
8690,Earth,False,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0
8691,Europa,False,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0


In [232]:
spaceship_test_data1

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,Earth,True,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0
1,Earth,False,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0
2,Europa,True,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0
3,Europa,False,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0
4,Earth,False,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...
4272,Earth,True,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,0.0,0.0
4273,Earth,False,TRAPPIST-1e,42.0,False,0.0,847.0,17.0,10.0,144.0
4274,Mars,True,55 Cancri e,NaN,False,0.0,0.0,0.0,0.0,0.0
4275,Europa,False,NaN,NaN,False,0.0,2680.0,0.0,0.0,523.0


In [233]:
data1 = pd.concat([spaceship_train_data1, spaceship_test_data1], axis=0).reset_index(drop=True)
data1

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0
1,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0
2,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0
3,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0
4,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0
...,...,...,...,...,...,...,...,...,...,...
12965,Earth,True,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,0.0,0.0
12966,Earth,False,TRAPPIST-1e,42.0,False,0.0,847.0,17.0,10.0,144.0
12967,Mars,True,55 Cancri e,NaN,False,0.0,0.0,0.0,0.0,0.0
12968,Europa,False,NaN,NaN,False,0.0,2680.0,0.0,0.0,523.0


In [234]:
#target
last_column

0       False
1        True
2       False
3       False
4        True
        ...  
8688    False
8689    False
8690     True
8691    False
8692     True
Name: Transported, Length: 8693, dtype: bool

In [235]:
data2 = data1.copy()

In [236]:
data2

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0
1,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0
2,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0
3,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0
4,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0
...,...,...,...,...,...,...,...,...,...,...
12965,Earth,True,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,0.0,0.0
12966,Earth,False,TRAPPIST-1e,42.0,False,0.0,847.0,17.0,10.0,144.0
12967,Mars,True,55 Cancri e,NaN,False,0.0,0.0,0.0,0.0,0.0
12968,Europa,False,NaN,NaN,False,0.0,2680.0,0.0,0.0,523.0


In [237]:
data2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12970 entries, 0 to 12969
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   HomePlanet    12682 non-null  object 
 1   CryoSleep     12660 non-null  object 
 2   Destination   12696 non-null  object 
 3   Age           12700 non-null  float64
 4   VIP           12674 non-null  object 
 5   RoomService   12707 non-null  float64
 6   FoodCourt     12681 non-null  float64
 7   ShoppingMall  12664 non-null  float64
 8   Spa           12686 non-null  float64
 9   VRDeck        12702 non-null  float64
dtypes: float64(6), object(4)
memory usage: 1013.4+ KB


In [238]:
from sklearn.compose import ColumnTransformer
from sklearn.base import TransformerMixin
from sklearn.base import BaseEstimator
from sklearn.impute import SimpleImputer
from sklearn.pipeline import Pipeline

#Changing all null numeric values to mean
class MultipleColumnImputerMean(BaseEstimator, TransformerMixin):
    
    def __init__(self, columns):
        self.columns = columns
    
    def fit(self, X, y=None):
        self.imputer = SimpleImputer(strategy='mean')
        self.imputer.fit(X[self.columns])
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        X_copy[self.columns] = self.imputer.transform(X_copy[self.columns])
        return X_copy

In [239]:
#Changing all null string values to mode

from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.impute import SimpleImputer

class MultipleColumnImputerMode(BaseEstimator, TransformerMixin):
    
    def __init__(self, columns):
        self.columns = columns
    
    def fit(self, X, y=None):
        self.imputer = SimpleImputer(strategy='most_frequent')
        self.imputer.fit(X[self.columns])
        return self
    
    def transform(self, X):
        X_copy = X.copy()
        X_copy[self.columns] = self.imputer.transform(X_copy[self.columns])
        return X_copy

In [240]:
#gets dummies/ one_hot_encoder
class GetDummies(BaseEstimator, TransformerMixin):
    
    def __init__(self, column_names):
        self.column_names = column_names

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        columns = X[self.column_names]  # Extract the specified columns

        # Apply one-hot encoding to each specified column
        dummies_list = [pd.get_dummies(columns[col], prefix=col) for col in self.column_names]

        # Concatenate one-hot encoded columns with the original DataFrame and delete dummied column
        X_encoded = pd.concat([X] + dummies_list, axis=1).drop(self.column_names, axis=1)

        return X_encoded

In [241]:
from sklearn.pipeline import Pipeline
pipeline = Pipeline([
    ('MultipleColumnImputerMean', MultipleColumnImputerMean(columns=['Age', 'RoomService','FoodCourt','ShoppingMall','Spa','VRDeck'])),
    ('MultipleColumnImputerMode', MultipleColumnImputerMode(columns=['HomePlanet','CryoSleep','Destination','VIP'])),
    ('getdummies', GetDummies(column_names=['HomePlanet','Destination']))])

In [242]:
data2

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck
0,Europa,False,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0
1,Earth,False,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0
2,Europa,False,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0
3,Europa,False,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0
4,Earth,False,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0
...,...,...,...,...,...,...,...,...,...,...
12965,Earth,True,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,0.0,0.0
12966,Earth,False,TRAPPIST-1e,42.0,False,0.0,847.0,17.0,10.0,144.0
12967,Mars,True,55 Cancri e,NaN,False,0.0,0.0,0.0,0.0,0.0
12968,Europa,False,NaN,NaN,False,0.0,2680.0,0.0,0.0,523.0


In [243]:
data3 = pipeline.fit_transform(data2)

In [244]:
data3

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,False,39.000000,False,0.0,0.0,0.0,0.0,0.0,0,1,0,0,0,1
1,False,24.000000,False,109.0,9.0,25.0,549.0,44.0,1,0,0,0,0,1
2,False,58.000000,True,43.0,3576.0,0.0,6715.0,49.0,0,1,0,0,0,1
3,False,33.000000,False,0.0,1283.0,371.0,3329.0,193.0,0,1,0,0,0,1
4,False,16.000000,False,303.0,70.0,151.0,565.0,2.0,1,0,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12965,True,34.000000,False,0.0,0.0,0.0,0.0,0.0,1,0,0,0,0,1
12966,False,42.000000,False,0.0,847.0,17.0,10.0,144.0,1,0,0,0,0,1
12967,True,28.771969,False,0.0,0.0,0.0,0.0,0.0,0,0,1,1,0,0
12968,False,28.771969,False,0.0,2680.0,0.0,0.0,523.0,0,1,0,0,0,1


In [245]:
data3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12970 entries, 0 to 12969
Data columns (total 14 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   CryoSleep                  12970 non-null  object 
 1   Age                        12970 non-null  float64
 2   VIP                        12970 non-null  object 
 3   RoomService                12970 non-null  float64
 4   FoodCourt                  12970 non-null  float64
 5   ShoppingMall               12970 non-null  float64
 6   Spa                        12970 non-null  float64
 7   VRDeck                     12970 non-null  float64
 8   HomePlanet_Earth           12970 non-null  uint8  
 9   HomePlanet_Europa          12970 non-null  uint8  
 10  HomePlanet_Mars            12970 non-null  uint8  
 11  Destination_55 Cancri e    12970 non-null  uint8  
 12  Destination_PSO J318.5-22  12970 non-null  uint8  
 13  Destination_TRAPPIST-1e    12970 non-null  uin

In [246]:
#Scaling¶

#Scaling¶

scaler = StandardScaler()
scaler.fit(data3)

data4 = pd.DataFrame(scaler.transform(data3), index=data3.index, columns=data3.columns)

In [247]:
data4

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,-0.738967,7.184543e-01,-0.146633,-0.347750,-0.288506,-0.299739,-0.275970,-0.262708,-1.108906,1.771949,-0.510820,-0.505656,-0.316951,0.646731
1,-0.738967,-3.352005e-01,-0.146633,-0.177696,-0.282761,-0.256896,0.215177,-0.225030,0.901790,-0.564350,-0.510820,-0.505656,-0.316951,0.646731
2,-0.738967,2.053084e+00,6.819762,-0.280664,1.994205,-0.299739,5.731407,-0.220749,-1.108906,1.771949,-0.510820,-0.505656,-0.316951,0.646731
3,-0.738967,2.969924e-01,-0.146633,-0.347750,0.530487,0.336049,2.702222,-0.097440,-1.108906,1.771949,-0.510820,-0.505656,-0.316951,0.646731
4,-0.738967,-8.971498e-01,-0.146633,0.124970,-0.243822,-0.040968,0.229491,-0.260996,0.901790,-0.564350,-0.510820,-0.505656,-0.316951,0.646731
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12965,1.353240,3.672360e-01,-0.146633,-0.347750,-0.288506,-0.299739,-0.275970,-0.262708,0.901790,-0.564350,-0.510820,-0.505656,-0.316951,0.646731
12966,-0.738967,9.291853e-01,-0.146633,-0.347750,0.252170,-0.270606,-0.267024,-0.139399,0.901790,-0.564350,-0.510820,-0.505656,-0.316951,0.646731
12967,1.353240,-2.495556e-16,-0.146633,-0.347750,-0.288506,-0.299739,-0.275970,-0.262708,-1.108906,-0.564350,1.957636,1.977630,-0.316951,-1.546237
12968,-0.738967,-2.495556e-16,-0.146633,-0.347750,1.422251,-0.299739,-0.275970,0.185144,-1.108906,1.771949,-0.510820,-0.505656,-0.316951,0.646731


In [248]:
#Split Data¶

train_final = data4.loc[:spaceship_train_data.index.max(), :].copy()
test_final = data4.loc[spaceship_train_data.index.max() + 1:, :].reset_index(drop=True).copy()

In [249]:
train_final

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,-0.738967,0.718454,-0.146633,-0.347750,-0.288506,-0.299739,-0.275970,-0.262708,-1.108906,1.771949,-0.51082,-0.505656,-0.316951,0.646731
1,-0.738967,-0.335201,-0.146633,-0.177696,-0.282761,-0.256896,0.215177,-0.225030,0.901790,-0.564350,-0.51082,-0.505656,-0.316951,0.646731
2,-0.738967,2.053084,6.819762,-0.280664,1.994205,-0.299739,5.731407,-0.220749,-1.108906,1.771949,-0.51082,-0.505656,-0.316951,0.646731
3,-0.738967,0.296992,-0.146633,-0.347750,0.530487,0.336049,2.702222,-0.097440,-1.108906,1.771949,-0.51082,-0.505656,-0.316951,0.646731
4,-0.738967,-0.897150,-0.146633,0.124970,-0.243822,-0.040968,0.229491,-0.260996,0.901790,-0.564350,-0.51082,-0.505656,-0.316951,0.646731
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,-0.738967,0.858942,6.819762,-0.347750,4.064349,-0.299739,1.193892,-0.199341,-1.108906,1.771949,-0.51082,1.977630,-0.316951,-1.546237
8689,1.353240,-0.756662,-0.146633,-0.347750,-0.288506,-0.299739,-0.275970,-0.262708,0.901790,-0.564350,-0.51082,-0.505656,3.155058,-1.546237
8690,-0.738967,-0.194713,-0.146633,-0.347750,-0.288506,2.908336,-0.275075,-0.262708,0.901790,-0.564350,-0.51082,-0.505656,-0.316951,0.646731
8691,-0.738967,0.226749,-0.146633,-0.347750,0.381115,-0.299739,0.039831,2.507469,-1.108906,1.771949,-0.51082,1.977630,-0.316951,-1.546237


In [250]:
train_final = pd.concat([train_final, last_column], axis=1)

In [251]:
train_final

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Transported
0,-0.738967,0.718454,-0.146633,-0.347750,-0.288506,-0.299739,-0.275970,-0.262708,-1.108906,1.771949,-0.51082,-0.505656,-0.316951,0.646731,False
1,-0.738967,-0.335201,-0.146633,-0.177696,-0.282761,-0.256896,0.215177,-0.225030,0.901790,-0.564350,-0.51082,-0.505656,-0.316951,0.646731,True
2,-0.738967,2.053084,6.819762,-0.280664,1.994205,-0.299739,5.731407,-0.220749,-1.108906,1.771949,-0.51082,-0.505656,-0.316951,0.646731,False
3,-0.738967,0.296992,-0.146633,-0.347750,0.530487,0.336049,2.702222,-0.097440,-1.108906,1.771949,-0.51082,-0.505656,-0.316951,0.646731,False
4,-0.738967,-0.897150,-0.146633,0.124970,-0.243822,-0.040968,0.229491,-0.260996,0.901790,-0.564350,-0.51082,-0.505656,-0.316951,0.646731,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,-0.738967,0.858942,6.819762,-0.347750,4.064349,-0.299739,1.193892,-0.199341,-1.108906,1.771949,-0.51082,1.977630,-0.316951,-1.546237,False
8689,1.353240,-0.756662,-0.146633,-0.347750,-0.288506,-0.299739,-0.275970,-0.262708,0.901790,-0.564350,-0.51082,-0.505656,3.155058,-1.546237,False
8690,-0.738967,-0.194713,-0.146633,-0.347750,-0.288506,2.908336,-0.275075,-0.262708,0.901790,-0.564350,-0.51082,-0.505656,-0.316951,0.646731,True
8691,-0.738967,0.226749,-0.146633,-0.347750,0.381115,-0.299739,0.039831,2.507469,-1.108906,1.771949,-0.51082,1.977630,-0.316951,-1.546237,False


In [252]:
train_final

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Transported
0,-0.738967,0.718454,-0.146633,-0.347750,-0.288506,-0.299739,-0.275970,-0.262708,-1.108906,1.771949,-0.51082,-0.505656,-0.316951,0.646731,False
1,-0.738967,-0.335201,-0.146633,-0.177696,-0.282761,-0.256896,0.215177,-0.225030,0.901790,-0.564350,-0.51082,-0.505656,-0.316951,0.646731,True
2,-0.738967,2.053084,6.819762,-0.280664,1.994205,-0.299739,5.731407,-0.220749,-1.108906,1.771949,-0.51082,-0.505656,-0.316951,0.646731,False
3,-0.738967,0.296992,-0.146633,-0.347750,0.530487,0.336049,2.702222,-0.097440,-1.108906,1.771949,-0.51082,-0.505656,-0.316951,0.646731,False
4,-0.738967,-0.897150,-0.146633,0.124970,-0.243822,-0.040968,0.229491,-0.260996,0.901790,-0.564350,-0.51082,-0.505656,-0.316951,0.646731,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,-0.738967,0.858942,6.819762,-0.347750,4.064349,-0.299739,1.193892,-0.199341,-1.108906,1.771949,-0.51082,1.977630,-0.316951,-1.546237,False
8689,1.353240,-0.756662,-0.146633,-0.347750,-0.288506,-0.299739,-0.275970,-0.262708,0.901790,-0.564350,-0.51082,-0.505656,3.155058,-1.546237,False
8690,-0.738967,-0.194713,-0.146633,-0.347750,-0.288506,2.908336,-0.275075,-0.262708,0.901790,-0.564350,-0.51082,-0.505656,-0.316951,0.646731,True
8691,-0.738967,0.226749,-0.146633,-0.347750,0.381115,-0.299739,0.039831,2.507469,-1.108906,1.771949,-0.51082,1.977630,-0.316951,-1.546237,False


In [253]:
test_final

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e
0,1.353240,-1.244695e-01,-0.146633,-0.347750,-0.288506,-0.299739,-0.275970,-0.262708,0.901790,-0.564350,-0.510820,-0.505656,-0.316951,0.646731
1,-0.738967,-6.864188e-01,-0.146633,-0.347750,-0.282761,-0.299739,2.249544,-0.262708,0.901790,-0.564350,-0.510820,-0.505656,-0.316951,0.646731
2,1.353240,1.565051e-01,-0.146633,-0.347750,-0.288506,-0.299739,-0.275970,-0.262708,-1.108906,1.771949,-0.510820,1.977630,-0.316951,-1.546237
3,-0.738967,6.482107e-01,-0.146633,-0.347750,3.957746,-0.299739,-0.114043,0.238236,-1.108906,1.771949,-0.510820,-0.505656,-0.316951,0.646731
4,-0.738967,-6.161751e-01,-0.146633,-0.332148,-0.288506,0.788470,-0.275970,-0.262708,0.901790,-0.564350,-0.510820,-0.505656,-0.316951,0.646731
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,1.353240,3.672360e-01,-0.146633,-0.347750,-0.288506,-0.299739,-0.275970,-0.262708,0.901790,-0.564350,-0.510820,-0.505656,-0.316951,0.646731
4273,-0.738967,9.291853e-01,-0.146633,-0.347750,0.252170,-0.270606,-0.267024,-0.139399,0.901790,-0.564350,-0.510820,-0.505656,-0.316951,0.646731
4274,1.353240,-2.495556e-16,-0.146633,-0.347750,-0.288506,-0.299739,-0.275970,-0.262708,-1.108906,-0.564350,1.957636,1.977630,-0.316951,-1.546237
4275,-0.738967,-2.495556e-16,-0.146633,-0.347750,1.422251,-0.299739,-0.275970,0.185144,-1.108906,1.771949,-0.510820,-0.505656,-0.316951,0.646731


In [254]:
#target

In [ ]:
from pycaret.classification import *

# Assuming 'train_final' is your DataFrame and 'target' is your target column
target_column_name = 'Transported'  # Replace with your actual target column name

# Setup PyCaret
s = setup(data=train_final, target=target_column_name, categorical_features={target_column_name: True})


In [199]:
from pycaret.classification import *

s = setup(train_final, target = target)

,Description,Value
0,Session id,8800
1,Target,Transported
2,Target type,Binary
3,Original data shape,"(8693, 15)"
4,Transformed data shape,"(8693, 15)"
5,Transformed train set shape,"(6085, 15)"
6,Transformed test set shape,"(2608, 15)"
7,Numeric features,14
8,Preprocess,True
9,Imputation type,simple


In [200]:
best = compare_models()

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
gbc,Gradient Boosting Classifier,0.7905,0.8688,0.8284,0.7727,0.7994,0.5807,0.5826,0.2890
lightgbm,Light Gradient Boosting Machine,0.7905,0.8641,0.8092,0.7825,0.7955,0.5808,0.5814,0.1910
ada,Ada Boost Classifier,0.7857,0.8597,0.8121,0.7741,0.7925,0.5712,0.5722,0.1560
lr,Logistic Regression,0.7834,0.8651,0.7906,0.7821,0.7862,0.5668,0.5670,7.6370
xgboost,Extreme Gradient Boosting,0.7791,0.8559,0.7945,0.7734,0.7837,0.5582,0.5585,0.4230
rf,Random Forest Classifier,0.7773,0.8486,0.7618,0.7892,0.7752,0.5548,0.5552,0.2990
svm,SVM - Linear Kernel,0.7735,0.0000,0.8013,0.7644,0.7801,0.5469,0.5512,0.0500
et,Extra Trees Classifier,0.7671,0.8339,0.7449,0.7829,0.7632,0.5344,0.5354,0.4000
ridge,Ridge Classifier,0.7581,0.0000,0.6888,0.8030,0.7412,0.5167,0.5223,0.1280
lda,Linear Discriminant Analysis,0.7579,0.8477,0.6888,0.8027,0.7411,0.5164,0.5220,0.0980


In [201]:
print(best)

GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='log_loss', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_samples_leaf=1,
                           min_samples_split=2, min_weight_fraction_leaf=0.0,
                           n_estimators=100, n_iter_no_change=None,
                           random_state=8800, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)


In [202]:
finalize_model(best)

Pipeline(memory=Memory(location=None),
         steps=[('numerical_imputer',
                 TransformerWrapper(exclude=None,
                                    include=['CryoSleep', 'Age', 'VIP',
                                             'RoomService', 'FoodCourt',
                                             'ShoppingMall', 'Spa', 'VRDeck',
                                             'HomePlanet_Earth',
                                             'HomePlanet_Europa',
                                             'HomePlanet_Mars',
                                             'Destination_55 Cancri e',
                                             'Destination_PSO J318.5-22',
                                             'Destination_TRAPPIST-1e'],
                                    transformer=SimpleImputer(ad...
                                            criterion='friedman_mse', init=None,
                                            learning_rate=0.1, loss='log_loss',
                                            max_depth=3, max_features=None,
                                            max_leaf_nodes=None,
                                            min_impurity_decrease=0.0,
                                            min_samples_leaf=1,
                                            min_samples_split=2,
                                            min_weight_fraction_leaf=0.0,
                                            n_estimators=100,
                                            n_iter_no_change=None,
                                            random_state=8800, subsample=1.0,
                                            tol=0.0001, validation_fraction=0.1,
                                            verbose=0, warm_start=False))],
         verbose=False)

In [203]:
evaluate_model(best)

interactive(children=(ToggleButtons(description='Plot Type:', icons=('',), options=(('Pipeline Plot', 'pipelin…

In [204]:
predict_model(best)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Gradient Boosting Classifier,0.7998,0.8889,0.8370,0.7811,0.8081,0.5995,0.6010


,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,Transported,prediction_label,prediction_score
1427,-0.738967,9.994289e-01,-0.146633,-3.477498e-01,-0.288506,1.785852,-0.274181,-0.262708,0.90179,-0.56435,-0.51082,-0.505656,-0.316951,0.646731,True,1,0.6611
6499,1.353240,1.601777e-02,-0.146633,-3.477498e-01,-0.288506,-0.299739,-0.275970,-0.262708,0.90179,-0.56435,-0.51082,-0.505656,3.155058,-1.546237,True,1,0.6758
7645,-0.738967,-1.388855e+00,-0.146633,-3.477498e-01,-0.288506,-0.299739,-0.275970,-0.262708,0.90179,-0.56435,-0.51082,-0.505656,-0.316951,0.646731,True,0,0.5296
8525,-0.738967,-5.459315e-01,-0.146633,-4.434158e-17,-0.288506,0.550264,0.108717,-0.262708,0.90179,-0.56435,-0.51082,-0.505656,-0.316951,0.646731,False,0,0.8013
8206,-0.738967,-8.269061e-01,-0.146633,-3.477498e-01,0.228551,-0.186634,-0.275970,-0.262708,0.90179,-0.56435,-0.51082,-0.505656,-0.316951,0.646731,True,1,0.6224
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5350,-0.738967,1.601777e-02,-0.146633,-3.477498e-01,-0.178073,-0.299739,-0.275970,0.726335,0.90179,-0.56435,-0.51082,-0.505656,3.155058,-1.546237,True,0,0.8836
3882,-0.738967,-2.495556e-16,-0.146633,-3.477498e-01,-0.007636,-0.299739,-0.275970,0.023301,0.90179,-0.56435,-0.51082,-0.505656,3.155058,-1.546237,False,0,0.6439
3736,-0.738967,-1.459099e+00,-0.146633,-3.477498e-01,-0.288506,-0.299739,-0.275970,-0.262708,0.90179,-0.56435,-0.51082,-0.505656,-0.316951,0.646731,True,0,0.5296
1128,-0.738967,5.077233e-01,-0.146633,-3.477498e-01,-0.288506,-0.299739,-0.275970,1.092837,0.90179,-0.56435,-0.51082,-0.505656,-0.316951,0.646731,False,0,0.9115


In [205]:
predictions = predict_model(best, test_final)

In [206]:
predictions

,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,prediction_label,prediction_score
0,1.353240,-1.244695e-01,-0.146633,-0.347750,-0.288506,-0.299739,-0.275970,-0.262708,0.901790,-0.564350,-0.510820,-0.505656,-0.316951,0.646731,1,0.6716
1,-0.738967,-6.864188e-01,-0.146633,-0.347750,-0.282761,-0.299739,2.249544,-0.262708,0.901790,-0.564350,-0.510820,-0.505656,-0.316951,0.646731,0,0.9230
2,1.353240,1.565051e-01,-0.146633,-0.347750,-0.288506,-0.299739,-0.275970,-0.262708,-1.108906,1.771949,-0.510820,1.977630,-0.316951,-1.546237,1,0.9722
3,-0.738967,6.482106e-01,-0.146633,-0.347750,3.957746,-0.299739,-0.114043,0.238236,-1.108906,1.771949,-0.510820,-0.505656,-0.316951,0.646731,1,0.9223
4,-0.738967,-6.161751e-01,-0.146633,-0.332148,-0.288506,0.788470,-0.275970,-0.262708,0.901790,-0.564350,-0.510820,-0.505656,-0.316951,0.646731,1,0.6219
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,1.353240,3.672360e-01,-0.146633,-0.347750,-0.288506,-0.299739,-0.275970,-0.262708,0.901790,-0.564350,-0.510820,-0.505656,-0.316951,0.646731,1,0.6716
4273,-0.738967,9.291853e-01,-0.146633,-0.347750,0.252170,-0.270606,-0.267024,-0.139399,0.901790,-0.564350,-0.510820,-0.505656,-0.316951,0.646731,0,0.5943
4274,1.353240,-2.495556e-16,-0.146633,-0.347750,-0.288506,-0.299739,-0.275970,-0.262708,-1.108906,-0.564350,1.957636,1.977630,-0.316951,-1.546237,1,0.9023
4275,-0.738967,-2.495556e-16,-0.146633,-0.347750,1.422251,-0.299739,-0.275970,0.185144,-1.108906,1.771949,-0.510820,-0.505656,-0.316951,0.646731,1,0.8196


In [207]:
target

'Transported'

In [208]:
test_ids

0       0013_01
1       0018_01
2       0019_01
3       0021_01
4       0023_01
         ...   
4272    9266_02
4273    9269_01
4274    9271_01
4275    9273_01
4276    9277_01
Name: PassengerId, Length: 4277, dtype: object

In [209]:
predictions['prediction_label']

0       1
1       0
2       1
3       1
4       1
       ..
4272    1
4273    0
4274    1
4275    1
4276    1
Name: prediction_label, Length: 4277, dtype: int32

In [210]:
Final_Results = pd.concat([test_ids, predictions], axis =1)

In [211]:
Final_Results

,PassengerId,CryoSleep,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,HomePlanet_Earth,HomePlanet_Europa,HomePlanet_Mars,Destination_55 Cancri e,Destination_PSO J318.5-22,Destination_TRAPPIST-1e,prediction_label,prediction_score
0,0013_01,1.353240,-1.244695e-01,-0.146633,-0.347750,-0.288506,-0.299739,-0.275970,-0.262708,0.901790,-0.564350,-0.510820,-0.505656,-0.316951,0.646731,1,0.6716
1,0018_01,-0.738967,-6.864188e-01,-0.146633,-0.347750,-0.282761,-0.299739,2.249544,-0.262708,0.901790,-0.564350,-0.510820,-0.505656,-0.316951,0.646731,0,0.9230
2,0019_01,1.353240,1.565051e-01,-0.146633,-0.347750,-0.288506,-0.299739,-0.275970,-0.262708,-1.108906,1.771949,-0.510820,1.977630,-0.316951,-1.546237,1,0.9722
3,0021_01,-0.738967,6.482106e-01,-0.146633,-0.347750,3.957746,-0.299739,-0.114043,0.238236,-1.108906,1.771949,-0.510820,-0.505656,-0.316951,0.646731,1,0.9223
4,0023_01,-0.738967,-6.161751e-01,-0.146633,-0.332148,-0.288506,0.788470,-0.275970,-0.262708,0.901790,-0.564350,-0.510820,-0.505656,-0.316951,0.646731,1,0.6219
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,1.353240,3.672360e-01,-0.146633,-0.347750,-0.288506,-0.299739,-0.275970,-0.262708,0.901790,-0.564350,-0.510820,-0.505656,-0.316951,0.646731,1,0.6716
4273,9269_01,-0.738967,9.291853e-01,-0.146633,-0.347750,0.252170,-0.270606,-0.267024,-0.139399,0.901790,-0.564350,-0.510820,-0.505656,-0.316951,0.646731,0,0.5943
4274,9271_01,1.353240,-2.495556e-16,-0.146633,-0.347750,-0.288506,-0.299739,-0.275970,-0.262708,-1.108906,-0.564350,1.957636,1.977630,-0.316951,-1.546237,1,0.9023
4275,9273_01,-0.738967,-2.495556e-16,-0.146633,-0.347750,1.422251,-0.299739,-0.275970,0.185144,-1.108906,1.771949,-0.510820,-0.505656,-0.316951,0.646731,1,0.8196


In [212]:
submission

,PassengerId,Transported
0,0013_01,1
1,0018_01,0
2,0019_01,1
3,0021_01,1
4,0023_01,1
...,...,...
4272,9266_02,1
4273,9269_01,0
4274,9271_01,1
4275,9273_01,1


In [213]:
submission['Transported'] = submission['Transported'].map({1: False, 0: True})


In [65]:
submission

,PassengerId,Transported
0,0013_01,False
1,0018_01,True
2,0019_01,False
3,0021_01,False
4,0023_01,False
...,...,...
4272,9266_02,False
4273,9269_01,True
4274,9271_01,False
4275,9273_01,False


In [66]:
submission = Final_Results[['PassengerId', 'prediction_label']].copy()

# Rename the column
submission = submission.rename(columns={'prediction_label': 'Transported'})

# Reset the indexes
submission.reset_index(drop=True, inplace=True)

In [68]:
submission.to_csv('C:\\Users\\Auditor\\Desktop\\Victor\\New folder\\Data Science\\Projects\\Final_submissionTitanicSpaceship23.csv'
, index=False, header=True)